## Experiment with cutting up models


In [8]:
from datetime import datetime
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *
import fastai
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from string import ascii_uppercase
import numpy as np
import operator
import glob

import torch
from torch import nn as nn
from torch.nn import functional as F


%matplotlib inline

# %matplotlib widget

plt.rcParams['figure.figsize'] = [9, 5]
plt.rcParams['figure.dpi'] = 120
plt.style.use('dark_background')

from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df.head()
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']

cat = Categorify()

to = TabularPandas(df, cat, cat_names)

cats = to.procs.categorify
cats['relationship']

to.show(max_n=3)




,workclass,education,marital-status,occupation,relationship,race
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White
1,Private,Masters,Divorced,Exec-managerial,Not-in-family,White
2,Private,HS-grad,Divorced,#na#,Unmarried,Black


In [9]:
to.cats.head()

,workclass,education,marital-status,occupation,relationship,race
0,5,8,3,0,6,5
1,5,13,1,5,2,5
2,5,12,1,0,5,3
3,6,15,3,11,1,2
4,7,6,3,9,6,3


In [10]:
norm = Normalize()
to = TabularPandas(df, norm, cont_names=cont_names)
norms = to.procs.normalize


In [11]:
norms.means


{'age': 38.58164675532078,
 'fnlwgt': 189778.36651208502,
 'education-num': 10.079815864562988}

In [12]:
norms.stds


{'age': 13.640223192304274,
 'fnlwgt': 105548.3568809906,
 'education-num': 2.573042731149292}

In [14]:
to.conts.head()


,age,fnlwgt,education-num
0,0.763796,-0.838084,0.746270
1,0.397233,0.444987,1.523560
2,-0.042642,-0.886734,NaN
3,-0.042642,-0.728873,1.912205
4,0.250608,-1.018314,NaN


In [15]:
fm = FillMissing(fill_strategy=FillStrategy.median)
to = TabularPandas(df, fm, cont_names=cont_names)
to.conts.head()


,age,fnlwgt,education-num
0,49,101320,12.0
1,44,236746,14.0
2,38,96185,10.0
3,38,112847,15.0
4,42,82297,10.0


In [16]:
to.cat_names


(#1) ['education-num_na']

In [17]:
to.cats.head()


,education-num_na
0,False
1,False
2,True
3,False
4,True


In [18]:
splits = RandomSplitter()(range_of(df))
splits

((#26049) [14860,30337,20177,23289,27833,28222,10731,775,9068,15458...],
 (#6512) [26459,14308,30129,7933,23325,4624,24726,19755,24006,30826...])

In [19]:
range_of(df)[:5], len(df)


([0, 1, 2, 3, 4], 32561)

In [20]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
y_names = 'salary'
y_block = CategoryBlock()

In [21]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, y_block=y_block, splits=splits)

In [22]:
dls = to.dataloaders()

In [23]:
trn_dl = TabDataLoader(to.train, bs=64, shuffle=True, drop_last=True)
val_dl = TabDataLoader(to.valid, bs=128)

In [25]:
dls = DataLoaders(trn_dl, val_dl)
dls.show_batch()


,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Self-emp-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,66.999999,111320.998827,13.0,<50k
1,Private,Some-college,Never-married,Sales,Own-child,White,False,17.999999,171087.999503,10.0,<50k
2,Private,HS-grad,Divorced,Sales,Not-in-family,White,False,35.000000,101387.000697,9.0,<50k
3,Federal-gov,Assoc-acdm,Never-married,Prof-specialty,Not-in-family,White,False,30.000000,43953.005629,12.0,<50k
4,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,33.000000,303867.000523,10.0,<50k
5,Local-gov,Masters,Divorced,Prof-specialty,Unmarried,White,False,52.999999,176557.000259,14.0,<50k
6,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,False,48.000000,188431.999985,9.0,>=50k
7,Private,Some-college,Never-married,Sales,Own-child,White,False,29.000000,147340.000639,10.0,<50k
8,Private,11th,Married-civ-spouse,Craft-repair,Husband,White,False,17.999999,115442.997762,7.0,<50k
9,Private,Some-college,Separated,Adm-clerical,Not-in-family,White,False,52.999999,263728.999184,10.0,<50k


In [26]:
to._dbunch_type == dls._dbunch_type


True

In [27]:
emb_szs = get_emb_sz(to); emb_szs


[(10, 6), (17, 8), (8, 5), (16, 8), (7, 5), (6, 4), (3, 3)]

In [28]:
to.cat_names


(#7) ['workclass','education','marital-status','occupation','relationship','race','education-num_na']

In [29]:
to['workclass'].nunique()


9

In [30]:
cont_len = len(to.cont_names); cont_len


3

In [31]:
batch = dls.one_batch(); len(batch)


3

In [32]:
batch[0][0], batch[1][0]


(tensor([1, 1, 3, 1, 1, 5, 1]), tensor([ 2.7517,  0.2054, -1.5989]))

In [33]:
net = TabularModel(emb_szs, cont_len, 2, [200,100])


In [34]:
net

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=42, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)

In [36]:
type(net)

fastai.tabular.model.TabularModel

In [39]:
net.children

<bound method Module.children of TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=42, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)>

In [41]:
dir(net.modules())


['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [42]:
net.modules()

<generator object Module.modules at 0x7f5831a86eb0>

In [44]:
type(net.layers)

torch.nn.modules.container.Sequential

In [45]:
net.layers.named_modules

<bound method Module.named_modules of Sequential(
  (0): LinBnDrop(
    (0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=42, out_features=200, bias=False)
    (2): ReLU(inplace=True)
  )
  (1): LinBnDrop(
    (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=200, out_features=100, bias=False)
    (2): ReLU(inplace=True)
  )
  (2): LinBnDrop(
    (0): Linear(in_features=100, out_features=2, bias=True)
  )
)>

In [48]:
net.layers[0][2] = Mish()

In [49]:
net.layers

Sequential(
  (0): LinBnDrop(
    (0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=42, out_features=200, bias=False)
    (2): Mish()
  )
  (1): LinBnDrop(
    (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=200, out_features=100, bias=False)
    (2): ReLU(inplace=True)
  )
  (2): LinBnDrop(
    (0): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [50]:
r = resnet18()

In [51]:
r.layer1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [66]:
r = xse_resnext50_deep()


In [67]:
r.named_children()

<generator object Module.named_children at 0x7f58a3854740>

In [71]:
r

XResNet(
  (0): ConvLayer(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convpath): Sequential(
        (0): ConvLayer(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [62]:
resnet18().modules


<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   